## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,WeatherPy_Database,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.65,115.33,66.99,36,41,6.06,scattered clouds
1,1,Vanavara,RU,60.34,102.28,70.52,38,95,7.34,overcast clouds
2,2,Upernavik,GL,72.79,-56.15,42.69,86,100,5.30,overcast clouds
3,3,Kaitangata,NZ,-46.28,169.85,41.00,72,68,11.01,broken clouds
4,4,Atuona,PF,-9.80,-139.03,78.53,76,10,14.07,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,WeatherPy_Database,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Atuona,PF,-9.80,-139.03,78.53,76,10,14.07,light rain
5,5,Kamina,CD,-8.74,24.99,83.73,35,0,0.89,clear sky
10,10,Bac Lieu,VN,9.29,105.72,87.49,69,43,13.13,moderate rain
11,11,Dakar,SN,14.69,-17.44,82.40,83,40,6.93,scattered clouds
13,13,Sur,OM,22.57,59.53,89.89,54,0,8.72,clear sky
25,25,Gizo,SB,-8.10,156.84,83.73,72,28,1.88,light rain
26,26,Douentza,ML,15.00,-2.95,78.89,68,98,4.03,overcast clouds
30,30,Saint-Leu,RE,-21.15,55.28,75.20,64,40,5.82,scattered clouds
32,32,Kamaishi,JP,39.27,141.88,87.80,55,75,11.41,broken clouds
34,34,Butaritari,KI,3.07,172.79,82.36,68,100,7.99,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

WeatherPy_Database     0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# No empty rows

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Atuona,PF,78.53,light rain,-9.80,-139.03,
5,Kamina,CD,83.73,clear sky,-8.74,24.99,
10,Bac Lieu,VN,87.49,moderate rain,9.29,105.72,
11,Dakar,SN,82.40,scattered clouds,14.69,-17.44,
13,Sur,OM,89.89,clear sky,22.57,59.53,
25,Gizo,SB,83.73,light rain,-8.10,156.84,
26,Douentza,ML,78.89,overcast clouds,15.00,-2.95,
30,Saint-Leu,RE,75.20,scattered clouds,-21.15,55.28,
32,Kamaishi,JP,87.80,broken clouds,39.27,141.88,
34,Butaritari,KI,82.36,overcast clouds,3.07,172.79,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
hotel_df['Hotel Name'].replace('',np.nan, inplace=True)
hotel_df.dropna()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Atuona,PF,78.53,light rain,-9.80,-139.03,Villa Enata
5,Kamina,CD,83.73,clear sky,-8.74,24.99,Guest House la Grace
10,Bac Lieu,VN,87.49,moderate rain,9.29,105.72,Khách sạn Sài Gòn Bạc Liêu
11,Dakar,SN,82.40,scattered clouds,14.69,-17.44,Novotel Dakar
13,Sur,OM,89.89,clear sky,22.57,59.53,Sur Plaza Hotel
...,...,...,...,...,...,...,...
663,Fallon,US,77.00,broken clouds,39.47,-118.78,Best Western Fallon Inn & Suites
668,Kitgum,UG,75.31,moderate rain,3.28,32.89,Jaflo Gardens Hotel
671,Vanimo,PG,82.90,scattered clouds,-2.67,141.30,Vahmoneh Lodge Vanimo
678,Tripoli,LY,81.07,clear sky,32.88,13.19,Al Waddan Hotel


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
max_temp = hotel_df["Max Temp"]
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
hotel_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))